# Майнор ВШЭ
## Прикладные задачи анализа данных 2020
## Обучение с подкреплением: CrossEntropy Method (CEM), Deep CEM.

## 1. Crossentropy Method

В этой пункте мы посмотрим на то, как решить задачи RL с помощью метода crossentropy.

Рассмотрим пример с задачей Taxi [Dietterich, 2000]. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

env = gym.make("Taxi-v3")
env.reset()
env.render()

In [2]:
n_states  = env.observation_space.n
n_actions = env.action_space.n  

print(f"состояний: {n_states} действий: {n_actions}")

В этот раз нашей стратегией будет вероятностной распределение. 

$\pi(s,a) = P(a|s)$

Для задачи такси мы можем использовать таблицу: 

policy[s,a] = P(выбрать действие a | в состоянии s)

Создадим "равномерную" стратегию в виде двумерного массива с 
равномерным распределением по действиям и сгенерируем игровую сессию с такой стратегией

In [3]:
def initialize_policy(n_states, n_actions):
    ####### Здесь ваш код ##########
    policy = np.array([[1./n_actions for _ in range(n_actions)] for _ in range(n_states)])
    ################################
    return policy

policy = initialize_policy(n_states, n_actions)

In [4]:
assert type(policy) in (np.ndarray, np.matrix)
assert np.allclose(policy, 1./n_actions)
assert np.allclose(np.sum(policy, axis=1), 1)

### Генерация сессий взаимодейтсвия со средой.

Мы будем запоминать все состояния, действия и вознаграждения за эпизод.

In [5]:
def generate_session(env, policy, t_max=10**4):
    """
    Игра идет до конца эпизода или до t_max шагов в окружении. 
    :param policy: [n_states,n_actions] 
    :returns: states - список состояний, actions - список действий, total_reward - итоговое вознаграждение
    """
    states, actions = [], []
    total_reward = 0.

    s = env.reset()

    for t in range(t_max):
        # Hint: you can use np.random.choice for sampling action
        # https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html
        # a = 
        ####### Здесь ваш код ########## 
        a = np.random.choice(n_actions, p=policy[s])
        ################################

        new_s, r, done, info = env.step(a)

        # Record information we just got from the environment.
        states.append(s)
        actions.append(a)
        total_reward += r

        s = new_s
        if done:
            break

    return states, actions, total_reward

In [6]:
s, a, r = generate_session(env, policy)
assert type(s) == type(a) == list
assert len(s) == len(a)
assert type(r) in [float, np.float]

In [7]:
# посмотрим на изначальное распределение вознаграждения
import matplotlib.pyplot as plt
%matplotlib inline

sample_rewards = [generate_session(env, policy, t_max=1000)[-1] for _ in range(200)]

plt.hist(sample_rewards, bins=20)
plt.vlines([np.percentile(sample_rewards, 50)], [0], [100], label="50'th percentile", color='green')
plt.vlines([np.percentile(sample_rewards, 90)], [0], [100], label="90'th percentile", color='red')
plt.legend()

### Реализация метода crossentropy  

Наша задача - выделить лучшие действия и состояния, т.е. такие, при которых было лучшее вознаграждение:

In [8]:
def select_elites(states_batch, actions_batch, 
                  rewards_batch, percentile=50):
    """
    Выбирает состояния и действия с заданным перцентилем (rewards >= percentile)
    :param states_batch: list of lists of states, states_batch[session_i][t]
    :param actions_batch: list of lists of actions, actions_batch[session_i][t]
    :param rewards_batch: list of rewards, rewards_batch[session_i]
    
    :returns: elite_states, elite_actions - одномерные 
    списки состояния и действия, выбранных сессий
    """
    # нужно найти порог вознаграждения по процентилю
    # reward_threshold =
    ####### Здесь ваш код ##########
    reward_threshold = np.percentile(rewards_batch, percentile)
    ################################
    
    
    # в соответствии с найденным порогом - отобрать 
    # подходящие состояния и действия
    # elite_states = 
    # elite_actions = 
    ####### Здесь ваш код ##########
    elite_states  = [session for session_i, session in enumerate(states_batch) if rewards_batch[session_i] >= reward_threshold]
    elite_states = [state for session in elite_states for state in session]
    
    elite_actions = [session for session_i, session in enumerate(actions_batch) if rewards_batch[session_i] >= reward_threshold]
    elite_actions = [action for session in elite_actions for action in session]
    ################################
    
    return elite_states, elite_actions

In [9]:
states_batch = [
    [1, 2, 3],     # game1
    [4, 2, 0, 2],  # game2
    [3, 1],        # game3
]

actions_batch = [
    [0, 2, 4],     # game1
    [3, 2, 0, 1],  # game2
    [3, 3],        # game3
]
rewards_batch = [
    3,  # game1
    4,  # game2
    5,  # game3
]

test_result_0 = select_elites(states_batch, actions_batch, rewards_batch, percentile=0)
test_result_30 = select_elites(states_batch, actions_batch, rewards_batch, percentile=30)
test_result_90 = select_elites(states_batch, actions_batch, rewards_batch, percentile=90)
test_result_100 = select_elites(states_batch, actions_batch, rewards_batch, percentile=100)

assert np.all(
    test_result_0[0] == [1, 2, 3, 4, 2, 0, 2, 3, 1]) \
       and np.all(
    test_result_0[1] == [0, 2, 4, 3, 2, 0, 1, 3, 3]), \
    "Для процентиля 0 необходимо выбрать все состояния " \
    "и действия в хронологическом порядке"

assert np.all(test_result_30[0] == [4, 2, 0, 2, 3, 1])\
   and np.all(test_result_30[1] == [3, 2, 0, 1, 3, 3]), \
    "Для процентиля 30 необходимо выбрать " \
    "состояния/действия из [3:]"
assert np.all(test_result_90[0] == [3, 1]) and \
       np.all(test_result_90[1] == [3, 3]), \
    "Для процентиля 90 необходимо выбрать состояния " \
    "действия одной игры"
assert np.all(test_result_100[0] == [3, 1]) and \
       np.all(test_result_100[1] == [3, 3]), \
    "Проверьте использование знаков: >=,  >. " \
    "Также проверьте расчет процентиля"
print("Тесты пройдены!")


Теперь мы хотим написать обновляющуюся стратегию:

In [10]:
def update_policy(elite_states,elite_actions):
    """
    Новой стратегией будет:
    policy[s_i,a_i] ~ #[вхождения  si/ai в лучшие states/actions]
    
    Не забудьте про нормализацию состояний.
    Если какое-то состояние не было посещено, 
    то используйте равномерное распределение 1./n_actions
    
    :param elite_states:  список состояний
    :param elite_actions: список действий
    """
    new_policy = np.zeros([n_states,n_actions])
    for state in range(n_states):
        # обновляем стратегию - нормируем новые частоты 
        # действий и не забываем про непосещенные состояния
        # new_policy[state, a] =         
        ####### Здесь ваш код ##########
        occur = 0
        for pos, s in enumerate(elite_states):
            if s == state:
                new_policy[state, elite_actions[pos]] += 1
                occur += 1
        if occur > 0:
            for a in range(n_actions):
                new_policy[state, a] /= occur
        else:
            for a in range(n_actions):
                new_policy[state, a] = 1./n_actions
        ################################
        
    return new_policy

In [11]:
elite_states, elite_actions = (
    [1, 2, 3, 4, 2, 0, 2, 3, 1],
    [0, 2, 4, 3, 2, 0, 1, 3, 3])

new_policy = update_policy(elite_states, elite_actions)

assert np.isfinite(
    new_policy).all(), "Стратегия не должна содержать " \
                       "NaNs или +-inf. Проверьте " \
                       "деление на ноль. "
assert np.all(
    new_policy >= 0), "Стратегия не должна содержать " \
                      "отрицательных вероятностей "
assert np.allclose(new_policy.sum(axis=-1),
                   1), "Суммарная\ вероятность действий"\
                       "для состояния должна равняться 1"
reference_answer = np.array([
    [1., 0., 0., 0., 0.],
    [0.5, 0., 0., 0.5, 0.],
    [0., 0.33333333, 0.66666667, 0., 0.],
    [0., 0., 0., 0.5, 0.5]])
assert np.allclose(new_policy[:4, :5], reference_answer)
print("Тесты пройдены!")

### Цикл обучения

Визуализириуем наш процесс обучения и также будем измерять распределение получаемых за сессию вознаграждений 

In [12]:
from IPython.display import clear_output

def show_progress(rewards_batch, log, percentile, reward_range=[-990, +10]):
    """
    Удобная функция, для визуализации результатов.
    """

    mean_reward = np.mean(rewards_batch)
    threshold = np.percentile(rewards_batch, percentile)
    log.append([mean_reward, threshold])
    
    plt.figure(figsize=[8, 4])
    plt.subplot(1, 2, 1)
    plt.plot(list(zip(*log))[0], label='Mean rewards')
    plt.plot(list(zip(*log))[1], label='Reward thresholds')
    plt.legend()
    plt.grid()

    plt.subplot(1, 2, 2)
    plt.hist(rewards_batch, range=reward_range)
    plt.vlines([np.percentile(rewards_batch, percentile)],
               [0], [100], label="percentile", color='red')
    plt.legend()
    plt.grid()
    clear_output(True)
    print("mean reward = %.3f, threshold=%.3f" % (mean_reward, threshold))
    plt.show()

In [13]:
# инициализируем стратегию
policy = initialize_policy(n_states, n_actions)

In [14]:
n_sessions = 250  # количество сессий для сэмплирования
percentile = 50  # перцентиль 
learning_rate = 0.5 # то как быстро стратегия будет обновляться 

log = []

for i in range(100):
    # генерируем n_sessions сессий
    # %time sessions = []
    ####### Здесь ваш код ##########
    sessions = [generate_session(env, policy) for _ in range(n_sessions)]
    ################################
    
    states_batch,actions_batch,rewards_batch = zip(*sessions)
    # отбираем лучшие действия и состояния ###
    # elite_states, elite_actions = 
    ####### Здесь ваш код ##########
    elite_states, elite_actions = select_elites(
    states_batch,actions_batch,rewards_batch, percentile)
    ################################
    
    # обновляем стратегию
    # new_policy =
    ####### Здесь ваш код ##########
    new_policy = update_policy(elite_states, elite_actions)
    ################################
    
    policy = learning_rate * new_policy + (1 - learning_rate) * policy

    # display results on chart
    show_progress(rewards_batch, log, percentile)

### Посмотрим на результаты
Задача такси быстро сходится, начиня с вознаграждения -1000 к почти оптимальному значению, а потом опять падает до -50/-100. Это вызвано случайностью в самом окружении - случайное начальное состояние пассажира и такси, в начале каждого эпизода. 

В случае если алгоритм CEM не сможет научиться тому, как решить задачу из какого-то стартового положения, он просто отбросит этот эпизод, т.к. не будет сессий, которые переведут этот эпизод в топ лучших. 

Для решения этой проблемы можно уменьшить threshold (порог лучших состояний) или изменить способ оценки стратегии, используя новую стратегию, полученную из каждого начального состояния и действия (теоретически правильный способ).

## 2. Deep CEM

В данной части мы рассмотрим применение CEM вместе с нейронной сетью.
Будем обучать многослойную нейронную сеть для решения простой задачи с непрерывным пространством действий.

<img src="https://camo.githubusercontent.com/8f39c7f54a7798e7f80c9ec5c0bb610696e5c5b7/68747470733a2f2f7469702e64756b652e6564752f696e646570656e64656e745f6c6561726e696e672f677265656b2f6c6573736f6e2f64696767696e675f6465657065725f66696e616c2e6a7067">

Будем тестировать нашего нового агента на известной задаче перевернутого маятника с непрерывным пространством состояний.
https://gym.openai.com/envs/CartPole-v0/

In [15]:
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

env = gym.make("CartPole-v0")

env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape[0]
                                        
print(f"состояний: {state_dim} действий: {n_actions}")

### Стратегия с нейронной сетью

Попробуем заменить метод обновления вероятностей на нейронную сеть. 
Будем пользоваться упрощенной реализацией нейронной сети из пакета Scikit-learn.
Нам потребуется: 
* agent.partial_fit(states, actions) - делает один проход обучения по данным. Максимизирует вероятность :actions: из :states:
* agent.predict_proba(states) - предсказыает вероятность каждого из действий, в виде матрицы размера [len(states), n_actions]

In [16]:
from sklearn.neural_network import MLPClassifier

agent = MLPClassifier(
    hidden_layer_sizes=(20, 20),
    activation='tanh',
)

# initialize agent to the dimension of state space and number of actions
agent.partial_fit([env.reset()] * n_actions, range(n_actions), range(n_actions))

In [17]:
def generate_session(env, agent, t_max=1000):
    
    states,actions = [],[]
    total_reward = 0
    
    s = env.reset()
    
    for t in range(t_max):
        # предсказываем вероятности действий по сети и 
        # выбираем одно действие
        # probs = 
        # a = 
        ####### Здесь ваш код ##########
        probs = agent.predict_proba([s])[0] 
        a = np.random.choice(n_actions, p=probs)
        ################################
        
        new_s,r,done,info = env.step(a)
        
        states.append(s)
        actions.append(a)
        total_reward+=r
        
        s = new_s
        if done: break
    return states,actions,total_reward

In [18]:
dummy_states, dummy_actions, dummy_reward = generate_session(env, agent, t_max=5)
print("состояния:", np.stack(dummy_states))
print("действия:", dummy_actions)
print("вознаграждение:", dummy_reward)

In [19]:
n_sessions = 100
percentile = 70
log = []

for i in range(100):
    ####### Здесь ваш код ##########
    sessions = [generate_session(env, agent) for _ in range(n_sessions)]
    ################################
    states_batch, actions_batch, rewards_batch = map(np.array, zip(*sessions))
    
    ####### Здесь ваш код ##########
    elite_states, elite_actions = select_elites(states_batch,actions_batch,rewards_batch, percentile)
    ################################
    
    # обновляем стратегию, для предсказания лучших состояний
    # elite_actions(y) из elite_states(X)
    ####### Здесь ваш код ##########
    agent.fit(elite_states, elite_actions)
    ################################
    
    show_progress(rewards_batch, log, percentile, reward_range=[0, np.max(rewards_batch)])

    if np.mean(rewards_batch) > 190:
        print("Принято!")
        break